In [1]:
import os, re, glob, requests, gzip, urllib.request, time, zipfile, shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm
from bs4 import BeautifulSoup

In [2]:
os.chdir('..')

---

read files

In [15]:
files = [f for f in os.listdir('/Users/maltemax/Desktop/files_sec') if f.endswith('zip')]

In [16]:
# with open(os.path.join(unzip_location, file, 'tag.tsv'), 'r') as f:
#     lis = [line.split('\t') for line in f]
# d = pd.DataFrame(lis)
# tag = pd.DataFrame(lis)
# tag.columns = tag.loc[0].tolist()
# tag = tag.drop(0)
# tag.replace(r'\n',' ', regex=True)

In [17]:
files_location = '/Users/maltemax/Desktop/files_sec'
unzip_location = '/Users/maltemax/Desktop/temp_unzip'
if not os.path.exists(unzip_location):
    os.makedirs(unzip_location)

collect = []

for file in tqdm(files):
    with zipfile.ZipFile(os.path.join(files_location, file), 'r') as zip_file:
        zip_file.extractall(os.path.join(unzip_location, file))
    
    # sub has all submissions for the given period
    sub = pd.read_csv(os.path.join(unzip_location, file, 'sub.tsv'), sep = '\t')
    sub = sub.loc[sub['form'] == '10-K']
    sub = sub[['adsh', 'cik', 'name', 'sic', 'period', 'fy', 'filed', 'instance']]
    
    # num has numerical values, one row for each tagged item
    num = pd.read_csv(os.path.join(unzip_location, file, 'num.tsv'), sep = '\t')
    num = num.loc[num['tag'].notna()]
    num['tag'] = num['tag'].str.lower()
    num = num.loc[num['tag'].str.contains('deferredtaxasset', flags = re.IGNORECASE)]
    
    # tag has tag information (official description, custom, etc.)
    tag = pd.read_csv(os.path.join(unzip_location, file, 'tag.tsv'), sep = '\t', on_bad_lines = 'warn')
    tag = tag.loc[tag['tag'].notna()]
    tag['tag'] = tag['tag'].str.lower()
    tag = tag.drop_duplicates('tag')
    tag = tag[['tag', 'custom', 'tlabel', 'doc']]
    
    # taking sub as the basis so that firms with no tags are retained
    # add num and tag information
    sub = sub.merge(right = num, how = 'left', on = 'adsh')
    sub = sub.merge(right = tag, how = 'left', on = 'tag')
    
    collect.append(sub)
    
    # delete the unzipped directory
    shutil.rmtree(os.path.join(unzip_location, file))

  0%|          | 0/74 [00:00<?, ?it/s]

/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_18369/1342275349.py:18: DtypeWarning: Columns (9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.path.join(unzip_location, file, 'num.tsv'), sep = '\t')
/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_18369/1342275349.py:18: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.path.join(unzip_location, file, 'num.tsv'), sep = '\t')
/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_18369/1342275349.py:18: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.path.join(unzip_location, file, 'num.tsv'), sep = '\t')
/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_18369/1342275349.py:18: DtypeWarning: Columns (9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.

/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_18369/1342275349.py:18: DtypeWarning: Columns (9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.path.join(unzip_location, file, 'num.tsv'), sep = '\t')
/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_18369/1342275349.py:18: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.path.join(unzip_location, file, 'num.tsv'), sep = '\t')
/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_18369/1342275349.py:24: ParserWarning: Skipping line 838521: expected 9 fields, saw 10

  tag = pd.read_csv(os.path.join(unzip_location, file, 'tag.tsv'), sep = '\t', on_bad_lines = 'warn')
/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_18369/1342275349.py:18: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.path.join(unzip_

In [33]:
df = pd.concat(collect, ignore_index = True)

/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_18369/3824935784.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(collect, ignore_index = True)


In [34]:
df['url_instance'] = ('https://www.sec.gov/Archives/edgar/data/' +
                      df['cik'].astype(str) + '/' +
                      df['adsh'].str.replace('-', '') + '/' +
                      df['instance'])

In [36]:
df.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'dta_data_raw.tsv'),
          sep = '\t', index = False)

---

In [100]:
df = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'dta_data_raw.tsv'),
                 sep = '\t')

In [101]:
df['tag'].value_counts().to_excel('test.xlsx')

In [102]:
tags = ['deferredtaxassetsgross',
        'deferredtaxassetsgrossnoncurrent',
        'deferredtaxassetsgrosscurrent',
        'deferredtaxassetgross']

In [103]:
df = df.loc[df['tag'].isin(tags)]

In [105]:
df['period'] = pd.to_datetime(df['period'], format = '%Y%m%d')
df['ddate'] = pd.to_datetime(df['ddate'], format = '%Y%m%d')

In [106]:
# calculate fiscal
df['fyear'] = np.where(df['period'].dt.month <= 5,
                       df['period'].dt.year - 1,
                       df['period'].dt.year)

In [107]:
df = df.loc[df['fyear'].isin(range(2012, 2024 + 1))]

In [108]:
df = df.loc[df['value'].notna()]

In [109]:
df['adsh'].nunique()

48327

In [110]:
# restrict to entries that are about the current fiscal year
df = df.loc[(df['period'].dt.year == df['ddate'].dt.year) &
            (df['period'].dt.month == df['ddate'].dt.month)]

In [112]:
df.loc[df['adsh'] == '0000353184-13-000009', ['tag', 'value']]

,tag,value
1059052,deferredtaxassetsgross,1002000.0
1059059,deferredtaxassetsgross,25000.0
1059065,deferredtaxassetsgross,227000.0


---

In [77]:
# replace INF (shown as 32767) by zero; INF means that it shows exact amounts, no rounding
# which is equal to dcml of 0
# some documents in the VA data also appear to have -32768, which I also set to 0
df['dcml'] = np.where((df['dcml'] == 32767) | (df['dcml'] == -32768), 0, df['dcml'])

In [78]:
# there are only very few cases with a dcml of > 0 (what do they mean?)
df = df.loc[df['dcml'] <= 0]

In [79]:
df['min_decimal'] = df.groupby('adsh')['dcml'].transform('min')

In [80]:
# round all amounts to lowest level of precision used in report
df['dta_min_accuracy'] = np.round(df['value'] / (10 ** abs(df['min_decimal'])), 0)

In [81]:
df = df.drop_duplicates(subset = ['cik', 'fyear', 'dta_min_accuracy'], keep = 'first')

---

In [82]:
def is_sum_of_numbers(number, numbers):
    if number == (sum(numbers) - number):
        return True
    else:
        return False

In [83]:
temp_dict = df.groupby('adsh')['dta_min_accuracy'].unique().to_dict()

In [84]:
dta_sum_of_others = {}

for k, v in temp_dict.items():
    dta_sum_of_others[k] = max([is_sum_of_numbers(number = n, numbers = v) for n in v])

In [85]:
dta_sum_of_others = pd.DataFrame(dta_sum_of_others, index = [0]).T.reset_index()

In [86]:
dta_sum_of_others.columns = ['adsh', 'dta_sum_of_others']

In [87]:
df = df.merge(right = dta_sum_of_others, how = 'left', on = 'adsh')

In [88]:
df['dta_max'] = df.groupby('adsh')['value'].transform('max')

In [89]:
df['drop'] = np.where((df['dta_sum_of_others'] == True) &
                      (df['value'] != df['dta_max']), 1, 0)

In [90]:
df = df.loc[df['drop'] == 0]

In [91]:
df = df.drop(columns = ['min_decimal', 'dta_min_accuracy', 'dta_max', 'dta_sum_of_others', 'drop'])

---

In [92]:
combos = [('deferredtaxassetsgrossnoncurrent',
           'deferredtaxassetsgrosscurrent')]

In [93]:
for combo in combos:
    # identify reports that have the two tags AND ONLY the two tags
    # otherwise I would drop companies that, e.g., only use
    # 'deferredtaxassetsvaluationallowancenoncurrent' as their only tag to tag VAs
    df['both_present'] = np.where((df['tag'] == combo[0]) |
                                  (df['tag'] == combo[1]), 1, 0)
    df['both_present'] = df.groupby('adsh')['both_present'].transform('sum')
    df['total_tags'] = df.groupby('adsh')['tag'].transform('nunique')
    save = df.loc[(df['both_present'] == 2) & (df['total_tags'] == 2)]
    
    # from va, drop unnecessary columns and drop those for which the tags will be summed up
    df = df.drop(columns = ['both_present', 'total_tags'])
    df = df.loc[~df['adsh'].isin(save['adsh'])]
    
    # sum up values and merge back info from 'save'
    aggregated_values = save.groupby('adsh')['value'].sum().reset_index()
    aggregated_values = aggregated_values.merge(right = save.drop_duplicates('adsh').drop(columns = ['both_present', 'total_tags', 'value']),
                                                on = 'adsh', how = 'left')
    # overwrite value of tag to be able to idenfity individually-calculated values later
    aggregated_values['tag'] = 'CALCULATED_SUM'
    
    # concatenate va (from which the values that are summed up individually have been dropped above) and
    # the individually-summed up values
    df = pd.concat([df, aggregated_values])

---

In [94]:
# drop all remaining duplicates
df = df.drop_duplicates(subset = ['cik', 'fyear'], keep = False)

In [95]:
df = df[['adsh', 'value']]

In [96]:
df['value'] = df['value'] / 1000000

In [97]:
df = df.rename(columns = {'value' : 'dta_gross'})

In [98]:
len(df)

46275

In [99]:
df.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'dta_gross.tsv'),
          sep = '\t', index = False)